#### Checking and upgrading the DKube SDK

In [ ]:
import pkg_resources, sys
try:
    dkube_sdk_version = float(pkg_resources.get_distribution("dkube").version)
except:
    dkube_sdk_version = 0 ## means the dkube sdk is not installed
if dkube_sdk_version < 3.7:
    !{sys.executable} -m pip install  git+https://github.com/oneconvergence/dkube.git@3.7 --user >/dev/null
%reset -f

#### DKube Resources

In [ ]:
import time,json
from dkube.sdk import *
import os
from urllib.parse import urlparse

# Set up font definitions for output
class style:
   RED = '\033[91m\033[1m'
   BOLD = '\033[1m'
   END = '\033[0m'

## Assign the username & token for access to the serving cluster
SERVING_DKUBE_USERNAME = os.getenv("DKUBE_USER_LOGIN_NAME","")
SERVING_DKUBE_TOKEN = os.getenv("DKUBE_USER_ACCESS_TOKEN","") 

#### User-Defined Variables

In [ ]:
## The monitor name will be the same as the deployment name.
## By default, it will be of the form <your user name>-chest-xray.
## If you want to create a monitor from a different deployment, provide the deployment name here.
MONITOR_NAME = f"{SERVING_DKUBE_USERNAME}-chest-xray"

## Required to train the model
DKUBE_TRAINING_CODE_NAME = "chest-xray"

## By default, you will be running this notebook script
##  (1) within the setup where the deployment is executing, and
##  (2) executing the monitor on the same cluster
## 
## If either of these are not true (i.e. you are running the monitor on another cluster),
## change the variable to SERVING_CLUSTER_EXECUTION = False
## You will then need to provide input to other variables as described below
SERVING_CLUSTER_EXECUTION = True

## If "SERVING_CLUSTER_EXECUTION = False" then SERVING_DKUBE_URL, SERVING_DKUBE_CLUSTER_NAME, & information on the
## monitoring cluster must be completed.
##
## The SERVING_DKUBE_URL is the external IP of the serving cluster (e.g. https://<External IP>:32222/)
##  Note: The final "/" must be included
SERVING_DKUBE_URL = ""


MONITORING_DKUBE_URL = ""
MONITORING_DKUBE_USERNAME = ""
MONITORING_DKUBE_TOKEN = ""

## Provide the existing cluster name (link) on the monitoring setup which points to the serving cluster.
## Leave it empty if a new cluster needs to be created.
## A new cluster will only be created if the user has operator role permission. 
SERVING_DKUBE_CLUSTER_NAME = ""

## Variables setup

In [ ]:
# If the monitor is on the same cluster as the serving, pick up everything by default.
# Otherwise, get the URL from the cell above
if SERVING_CLUSTER_EXECUTION:
    MINIO_ENDPOINT = os.getenv("MLFLOW_S3_ENDPOINT_URL")
    SERVING_DKUBE_URL = os.getenv("DKUBE_URL")
else:
    parsed_url = urlparse(SERVING_DKUBE_URL)
    SERVING_DKUBE_IP = parsed_url.hostname
    MINIO_ENDPOINT = f"http://{SERVING_DKUBE_IP}:32221"

if (not SERVING_DKUBE_URL) or (not MINIO_ENDPOINT):
    raise ValueError("Either SERVING_DKUBE_URL or MINIO_ENDPOINT is empty")

# Required in train.ipynb to retrain the model
LIVE_DATASET =  MONITOR_NAME+'-s3'
TRAINING_DATASET = 'chest-xray'

# Model name to be created or used for example, it will create the model
# if not existing otherwise it will ignore creation.
MODEL_NAME = MONITOR_NAME

# the frequency with which monitoring will run, value will be considered in minutes
RUN_FREQUENCY = 5


if not MODEL_NAME:
    raise Exception("Model name is empty")

if not(SERVING_DKUBE_TOKEN and SERVING_DKUBE_USERNAME and SERVING_DKUBE_URL):
    raise Exception("Please fill the Serving Dkube details first (SERVING_DKUBE_TOKEN, SERVING_DKUBE_URL, SERVING_DKUBE_USERNAME)")
    
if (MONITORING_DKUBE_URL
   and
   not(MONITORING_DKUBE_USERNAME and MONITORING_DKUBE_TOKEN)):
    raise Exception("Please fill the Monitoring Dkube details first (MONITORING_DKUBE_USERNAME, MONITORING_DKUBE_TOKEN, MONITORING_DKUBE_URL)")

#### General Variable Definitions

In [ ]:
serving_api = DkubeApi(URL=SERVING_DKUBE_URL,token=SERVING_DKUBE_TOKEN)
if SERVING_DKUBE_USERNAME == serving_api.validate_token()['username']:
    pass
else:
    print("Invalid User, please check your username, first")

In [ ]:
response =  serving_api.get_cloudevents_logstore_creds()
MINIO_KEY = response["access_key_id"]
MINIO_SECRET_KEY = response["access_key"]
MINIO_BUCKET = response["bucket"]

In [ ]:
INPUT_TRAIN_TYPE = "training"
image_exp_config = {"MONITOR_NAME":MONITOR_NAME,
                    "INPUT_TRAIN_TYPE":INPUT_TRAIN_TYPE, "SERVING_DKUBE_USERNAME":SERVING_DKUBE_USERNAME,
                    "SERVING_DKUBE_URL":SERVING_DKUBE_URL, "SERVING_DKUBE_TOKEN":SERVING_DKUBE_TOKEN, 
                    "MINIO_KEY":MINIO_KEY,"MINIO_SECRET_KEY":MINIO_SECRET_KEY, "MINIO_ENDPOINT":MINIO_ENDPOINT,
                    "MINIO_BUCKET": MINIO_BUCKET,
                    "TRAINING_DATASET":TRAINING_DATASET, "RUN_FREQUENCY":RUN_FREQUENCY,
                    "DKUBE_TRAINING_CODE_NAME":DKUBE_TRAINING_CODE_NAME,
                    "LIVE_DATASET":LIVE_DATASET,
                    "MONITORING_DKUBE_USERNAME":MONITORING_DKUBE_USERNAME, "MONITORING_DKUBE_TOKEN":MONITORING_DKUBE_TOKEN,
                    "MONITORING_DKUBE_URL":MONITORING_DKUBE_URL,
                    "SERVING_DKUBE_CLUSTER_NAME":SERVING_DKUBE_CLUSTER_NAME}
%store image_exp_config

#### Create Training Dataset on Serving Cluster

In [ ]:
try:
    print(f"{style.BOLD}Creating Training Dataset \"{TRAINING_DATASET}\" on Serving Cluster{style.END}")
    dataset = DkubeDataset(SERVING_DKUBE_USERNAME, name=TRAINING_DATASET)
    dataset.update_dataset_source(source="git")
    dataset.update_git_details(url="https://github.com/oneconvergence/dkube-examples/tree/monitoring/image_cloudevents/data/chest-xray-mini")
    serving_api.create_dataset(dataset)
except Exception as e:
    if e.reason.lower()!="conflict":
        response = e.body
        print(f"Failed[{response.code}]: {response.message}")
    else:
        print(f"{style.BOLD}Training Dataset \"{TRAINING_DATASET}\" exists on Serving Cluster{style.END}")

#### Create Model on Serving Cluster

In [ ]:
print(f"{style.BOLD}Creating Model \"{MODEL_NAME}\" on Serving Cluster{style.END}")
try:
    model = DkubeModel(SERVING_DKUBE_USERNAME, name=MODEL_NAME)
    model.update_model_source(source='dvs')
    serving_api.create_model(model)
except Exception as e:
    if e.reason:
        if e.reason.lower() != "conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")
        else:
            print(f"{style.BOLD}Model \"{MODEL_NAME}\" exists on Serving Cluster{style.END}")
    else:
        raise e

#### Checking for seperate monitoring cluster and adding datasets accordingly.

In [ ]:
if MONITORING_DKUBE_URL:
    print(f"{style.BOLD}Creating Training Dataset \"{TRAINING_DATASET}\" on Monitoring Cluster{style.END}")
    monitoring_api = DkubeApi(URL=MONITORING_DKUBE_URL,token=MONITORING_DKUBE_TOKEN)
    try:
        dataset = DkubeDataset(MONITORING_DKUBE_USERNAME, name=TRAINING_DATASET)
        dataset.update_dataset_source(source="git")
        dataset.update_git_details(url="https://github.com/oneconvergence/dkube-examples/tree/monitoring/image_cloudevents/data/chest-xray-mini")
        serving_api.create_dataset(dataset)
    except Exception as e:
        if e.reason.lower()!="conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")
        else:
            print(f"{style.BOLD}Training Dataset \"{TRAINING_DATASET}\" exists on Monitoring Cluster{style.END}")


### Model Monitor Datasets

##### Predict and labelled Dataset

In [ ]:
# Set up the right cluster for the predict and labelled dataset
if MONITORING_DKUBE_URL:
    api = monitoring_api  
    user = MONITORING_DKUBE_USERNAME
else:
    api = serving_api
    user = SERVING_DKUBE_USERNAME

In [ ]:
data = api.get_modelmonitor_id(MONITOR_NAME)
if data.data:
    MONITOR_ID = data.data.get(MONITOR_NAME)
    if MONITOR_ID:
        raise ValueError(f"{MONITOR_NAME} already exists. Please use a different name")

In [ ]:
# Create the predict & label dataset on the serving or monitoring cluster
try:
    dataset = DkubeDataset(user, name=LIVE_DATASET,remote=True)
    dataset.update_dataset_source('s3')
    dataset.update_s3_details(
        endpoint = MINIO_ENDPOINT,
        bucket=MINIO_BUCKET,
        prefix='',
        key=MINIO_KEY,
        secret=MINIO_SECRET_KEY)
    
    if MONITORING_DKUBE_URL:
        cluster_type = "Monitoring"
    else:
        cluster_type = "Serving"
    print(f"{style.BOLD}Creating Live Dataset \"{LIVE_DATASET}\" on {cluster_type} Cluster{style.END}")

    api.create_dataset(dataset)
except Exception as e:
    if e.reason:
        if e.reason.lower() != "conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")
        else:
            print(f"{style.BOLD}Live Dataset \"{LIVE_DATASET}\" exists on {cluster_type} Cluster{style.END}")
    else:
        raise e

#### Cleanup

In [ ]:
## Set CLEANUP = True, after your experiment is complete.
CLEANUP = False
if CLEANUP:
    # Delete the serving cluster resources
    api = serving_api
    api.delete_model(SERVING_DKUBE_USERNAME,MODEL_NAME,force=True)        
    api.delete_dataset(SERVING_DKUBE_USERNAME,TRAINING_DATASET,force=True)        

    # Delete the monitoring cluster resources if separate cluster
    if MONITORING_DKUBE_URL:
        api = monitoring_api
        api.delete_dataset(SERVING_DKUBE_USERNAME,TRAINING_DATASET,force=True)
        api.delete_dataset(SERVING_DKUBE_USERNAME,LIVE_DATASET,force=True)

    # Or delete the live dataset on the serving cluster
    else:
        api = serving_api
        api.delete_dataset(SERVING_DKUBE_USERNAME,LIVE_DATASET,force=True)        

    %store -d image_exp_config